In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Peer-graded Assignment: Описательный анализ данных

Часто при работе с пользовательскими данными приходится сталкиваться с тем, что данные предоставляются для работы без описания. При этом не известно, что содержательно представляют собой те или иные признаки, а известны лишь их типы: числовые, категориальные, номинальные переменные. Такие ситуации - не редкость при работе с «чувствительными» данными, например, в сфере банковской аналитики, HR-аналитики, сфере телекоммуникаций, страхования, здравоохранения, недвижимости или ритейла. Тем не менее, с такими данным нужно уметь работать, и нужно уметь проводить на них классические этапы анализа, в частности описательный анализ данных и визуализацию. Именно этим мы займемся на первой неделе.


В этом задании мы потренируемся делать описательный анализ и визуализацию так называемых «закрытых» данных - данных, которые предоставляются для анализа и построения моделей без описания. Выборка, с которой мы будем работать прилагается.

* Метки классов (здесь `1` соответствует классу `отток`, `-1` - классу `не отток`) – `orange_small_churn_labels.train`
* Данные (предоставлены без описания, однако известно, что первые 190 переменных являются числовыми, а оставшиеся 40 - категориальными) – `orange_small_churn_data.train`

Скачайте эти файлы [на странице задания](https://www.coursera.org/learn/data-analysis-project/peer/roSOe/opisatiel-nyi-analiz-dannykh) и переходите к выполнению задачи! Результатом выполнения задания является jupyter notebook. Перед отправкой результата убедитесь, что вы выполнили все пункты из задания и это можно понять из вашего notebook'а. Проверьте, что код написан понятно и аккуратно - это поможет вашим сокурсником лучше понять, что вы сделали, и оценить вашу работу.

## Инструкции

1\. Загрузите данные `orange_small_churn_data.train` и `orange_small_churn_labels.train`

In [80]:
data = pd.read_csv('orange_small_churn_data.txt')
labels = pd.read_csv('orange_small_churn_labels.txt', header = None, names=['label'])
data = pd.concat([data,labels], axis=1)
print(data.shape)
data.head(4)

(40000, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1


In [92]:
# Удалим все столбцы, содержащие только NaN
for var in data.columns:
    if data[var].isna().sum() == data.shape[0]:
        data.drop(var, axis=1,inplace=True)
# В категориальных признаках заменим пропуски на новую категорию
data.loc[:,'Var191':] = data.loc[:,'Var191':].fillna('new_category')

In [93]:
# data.isna().sum()

2\. Рассчитайте доли классов `отток` и `не отток`.

In [94]:
not_churn = data.label.value_counts()[-1]
churn = data.label.value_counts()[1]

print('Доля класса "отток": ', churn/data.shape[0])
print('Доля класса "не отток": ', not_churn/data.shape[0])

Доля класса "отток":  0.0744
Доля класса "не отток":  0.9256


3\. Рассчитайте корреляции переменных с целевой функцией и проанализируйте полученные данные.

Обратите внимание на то, что целевая функция предсталвляет собой бинарную переменную. Теоретически, это не помешает рассчиать, например, корреляцию Пирсона между непрерывной переменной и целевой функцией, онднако анализиоовать полученные результаты будет сложно, корреляция Пирсона просто не рассчитана на такое применение. Кто уже забыл, как действовать в такой ситуации - вспоминить можно вот здесь: https://www.coursera.org/teach/stats-for-data-analysis/content/edit/lecture/prX3S/video-subtitles 

Из лекции: 

Для пар признаков, один из которых непрерывный, а другой — категориальный, вообще не
нужно считать никакой коэффициент корреляции.

$ X1 ∈ R$  и $X2 ∈ {0, 1} $ будут положительно коррелированы,если

$E (X1 | X2 = 1) > E (X1 | X2 = 0)$.
        
Таким образом, мерой силы взаимосвязи между $X1$ и $X2$ может служить просто разность этих математических ожиданий:
$E (X1 | X2 = 1) − E (X1 | X2 = 0)$

Эта величина не нормированная, она может меняться в любом диапазоне, от $−∞$ до $+∞$. Однако её гораздо
легче интерпретировать, чем коэффициент корреляции, который можно вычислить на такой паре выборок.


In [100]:
# Посчитаем корреляции для числовых признаков
numeric_corrs = []
for var in data.loc[:,:'Var190'].columns:
    var_churn = data.loc[data.label == 1,var]
    var_not_churn = data.loc[data.label == -1,var]
    numeric_corrs.append( [var, var_churn.mean() - var_not_churn.mean()] )
numeric_corrs = np.array(numeric_corrs)

In [97]:
# Для расчета корреляции между целевым признаком и категориальными переменными,
# используем коэффициент V Крамера. Для этого составим таблицу сопряженности

for var in data.loc[:,'Var191':].columns:
    print(var,len(data[var].unique()), data[var].isna().sum())


Var191 2 0
Var192 355 0
Var193 50 0
Var194 4 0
Var195 23 0
Var196 4 0
Var197 221 0
Var198 3891 0
Var199 4401 0
Var200 13298 0
Var201 3 0
Var202 5543 0
Var203 6 0
Var204 100 0
Var205 4 0
Var206 22 0
Var207 14 0
Var208 3 0
Var210 6 0
Var211 2 0
Var212 78 0
Var213 2 0
Var214 13298 0
Var215 2 0
Var216 1819 0
Var217 12471 0
Var218 3 0
Var219 23 0
Var220 3891 0
Var221 7 0
Var222 3891 0
Var223 5 0
Var224 2 0
Var225 4 0
Var226 23 0
Var227 7 0
Var228 30 0
Var229 5 0
label 2 0


4\. Визуализируйте данные: рассмотрите топ 20 числовых переменных, наиболее сильно коррелирующих с целевой функцией. Для этих переменных постройте:
* Распределения в разрезе классов
* Отобразите объекты в координатах пар признаков

5\. Проделайте аналогичные шаги для случайно выбранных 10 числовых признаков.

6\. Проделайте аналогичные шаги для 10 числовых признаков, наименее сильно коррелирующих с целевой переменной.

7\. Посмотрите на категориальные переменные: постройте и сравните гистограммы значений категориальных переменных для разных классов.

8\. Проанализируйте полученные результаты:
* Какие выводы вы можете сделать? Заметили ли вы какие-нибудь интересные закономерности?
* На основании полученных изображений и таблиц, предположите, какие переменные окажут наибольшее влияние (вклад) в модель?
* Какие переменные на ваш взгляд окажутся наименее полезными, шумовыми?
* Как вы думаете, окажется ли отбор признаков полезным для построения модели?